In [2]:
OPENAI_API_BASE_URL="https://api.chatanywhere.tech/v1"
OPENAI_API_KEY="sk-5aIvPvcD0VPm2QePq6Y2cnKN0DQIxIlq9HQgQQPYhGSlBATN"
TAVILY_API_KEY="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"

In [12]:
"""Main entrypoint for the app."""
import asyncio
import os
from datetime import datetime
from operator import itemgetter
from typing import List, Optional, Sequence, Tuple, Union
import langsmith
from fastapi import FastAPI, Request, Depends
from fastapi.middleware.cors import CORSMiddleware
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.chat_models import ChatAnthropic, ChatOpenAI, ChatVertexAI
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.retrievers import (
    ContextualCompressionRetriever,
    TavilySearchAPIRetriever,
)
from langchain.retrievers.document_compressors import (
    DocumentCompressorPipeline,
    EmbeddingsFilter,
)
from langchain.retrievers.kay import KayAiRetriever
from langchain.retrievers.you import YouRetriever
from langchain.schema import Document
from langchain.schema.document import Document
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.retriever import BaseRetriever
from langchain.schema.runnable import (
    ConfigurableField,
    Runnable,
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter

RESPONSE_TEMPLATE = """\

You are an expert researcher and writer, tasked with answering any question.

Generate a comprehensive and informative, yet concise answer of 250 words or less for the \
given question based solely on the provided search results (URL and content). You must \
only use information from the provided search results. Use an unbiased and \
journalistic tone. Combine search results together into a coherent answer. Do not \
repeat text. Cite search results using [${{number}}] notation. Only cite the most \
relevant results that answer the question accurately. Place these citations at the end \
of the sentence or paragraph that reference them - do not put them all at the end. If \
different results refer to different entities within the same name, write separate \
answers for each entity. If you want to cite multiple results for the same sentence, \
format it as `[${{number1}}] [${{number2}}]`. However, you should NEVER do this with the \
same number - if you want to cite `number1` multiple times for a sentence, only do \
`[${{number1}}]` not `[${{number1}}] [${{number1}}]`


<context>
    {context}
<context/>

The current date is {current_date}.
"""

REPHRASE_TEMPLATE = """\
Given the following conversation and a follow up question, rephrase the follow up \
question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone Question:"""


base_tavily_retriever = TavilySearchAPIRetriever(
    k=2,
    api_key=TAVILY_API_KEY
)

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    # model="gpt-4",
    streaming=True,
    temperature=0.1,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE_URL,
)

def create_retriever_chain(
    llm: BaseLanguageModel, retriever: BaseRetriever
) -> Runnable:
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(REPHRASE_TEMPLATE)
    condense_question_chain = (
        CONDENSE_QUESTION_PROMPT | llm | StrOutputParser()
    ).with_config(
        run_name="CondenseQuestion",
    )
    conversation_chain = condense_question_chain | retriever
    return RunnableBranch(
        (
            RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
                run_name="HasChatHistoryCheck"
            ),
            conversation_chain.with_config(run_name="RetrievalChainWithHistory"),
        ),
        (
            RunnableLambda(itemgetter("question")).with_config(
                run_name="Itemgetter:question"
            )
            | retriever
        ).with_config(run_name="RetrievalChainWithNoHistory"),
    ).with_config(run_name="RouteDependingOnChatHistory")

chain = create_retriever_chain(llm, base_tavily_retriever)

chain.invoke({"question":"what is the weather today in Beijing?"})

[Document(page_content="{'location': {'name': 'Beijing', 'region': 'Beijing', 'country': 'China', 'lat': 39.9289, 'lon': 116.3883, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1732726518, 'localtime': '2024-11-28 00:55'}, 'current': {'last_updated_epoch': 1732725900, 'last_updated': '2024-11-28 00:45', 'temp_c': 1.4, 'temp_f': 34.5, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 15.0, 'wind_kph': 24.1, 'wind_degree': 309, 'wind_dir': 'NW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 29, 'cloud': 0, 'feelslike_c': -4.0, 'feelslike_f': 24.8, 'windchill_c': -4.2, 'windchill_f': 24.4, 'heatindex_c': 1.3, 'heatindex_f': 34.3, 'dewpoint_c': -15.0, 'dewpoint_f': 4.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 21.9, 'gust_kph': 35.2}}", metadata={'title': 'Weather in Beijing', 'source': 'https://www.weatherapi.com/', 'score': 0.9991955, 'images': []